# Airbnb Dynamic Pricing Analysis

This notebook demonstrates the **Economic Demand Curve** pricing engine using the Kaggle European Cities dataset.

### Objectives
1. **Estimate Demand**: Model booking probability as a function of price.
2. **Maximize Revenue**: Find the optimal price point *P\** where revenue *R = P × D(P)* is maximized.
3. **Analyze Segments**: Compare optimal prices across different cities.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add scripts to path
sys.path.append('../scripts')
from verify_logic import estimate_market_parameters, find_optimal_price, exponential_demand

%matplotlib inline
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data
We use the processed Kaggle dataset containing 50k+ listings from 10 major European cities.

In [ ]:
DATA_PATH = '../data/processed/europe_airbnb.csv'

if not os.path.exists(DATA_PATH):
    print("Data not found! Please run `python scripts/process_kaggle_data.py` first.")
else:
    df = pd.read_csv(DATA_PATH)
    print(f"Loaded {len(df):,} listings from {len(df['city'].unique())} cities.")
    display(df.head())

## 2. Market Parameter Estimation
We estimate the **Base Demand** ($D_0$) and **Price Sensitivity** ($\lambda$) from the market data.

In [ ]:
base_demand, sensitivity, ref_price = estimate_market_parameters(df)

print(f"Market Reference Price: ${ref_price:.2f}")
print(f"Base Demand (at ref price): {base_demand:.1%}")
print(f"Price Sensitivity (Elasticity Factor): {sensitivity:.2f}")

## 3. Revenue Optimization
We search for the price that maximizes Expected Revenue:
$$ E[R] = P \times D(P) $$
where $D(P) = D_0 \cdot e^{-\lambda \frac{P - P_{ref}}{P_{ref}}}$

In [ ]:
# Run Optimization
opt_price, opt_prob, max_rev, elasticity, price_range, demands = \
    find_optimal_price(base_demand, sensitivity, ref_price, min_price=30, max_price=500)

revenues = price_range * demands

print(f"OPTIMAL PRICE: ${opt_price:.2f}")
print(f"Booking Probability: {opt_prob:.1%}")
print(f"Expected Revenue: ${max_rev:.2f}/night")

## 4. Visualization
Visualizing the Demand Curve and Revenue outcomes.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Demand Curve
ax1.plot(price_range, demands * 100, linewidth=2.5)
ax1.axvline(opt_price, color='green', linestyle='--', label=f'Optimal: ${opt_price:.0f}')
ax1.set_title(f'Demand Curve (Elasticity: {elasticity:.2f})', fontsize=14)
ax1.set_xlabel('Price ($)')
ax1.set_ylabel('Booking Probability (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Revenue Curve
ax2.plot(price_range, revenues, color='purple', linewidth=2.5)
ax2.axvline(opt_price, color='green', linestyle='--', label=f'Max Rev: ${max_rev:.0f}')
ax2.set_title('Revenue Optimization Surface', fontsize=14)
ax2.set_xlabel('Price ($)')
ax2.set_ylabel('Expected Revenue ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()